In [1]:
from IPython.display import HTML
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import precision_score

import urllib2
import bs4
import time
import operator
import re 
    
%matplotlib inline
import seaborn as sns

In [2]:
#Time Series Data
UR = pd.read_csv('UnempRate.csv', sep=',', na_values=[" ", ""])
crime = pd.read_csv('violent_crime_count_age17andup.csv', sep=',', na_values=[" ", ""])
#prescription is per 100
prescription = pd.read_csv('prescription_rate.csv', sep=',', na_values=[" ", ""])
population = pd.read_csv('population.csv', sep=',', na_values=[" ", ""])
birth_rate = pd.read_csv('birth_rate.csv', sep=',', na_values=[" ", ""])
death_rate = pd.read_csv('death_rate.csv', sep=',', na_values=[" ", ""])

In [3]:
birth_rate.iloc[:,1:]=birth_rate.iloc[:,1:]/1000
death_rate.iloc[:,1:]=death_rate.iloc[:,1:]/1000

In [6]:
#Non-Time Series Data
data = pd.read_csv('cleaned_GaCountiesODDeath.csv', na_values=[" ", ""] ,sep=',', index_col=0)
hospital = pd.read_csv('cleaned_GaHospitals.csv', na_values=[" ", ""] ,sep=',', index_col=0).fillna(0)
dropbox = pd.read_csv('drug_dropbox.csv', na_values=[" ", ""] ,sep=',').fillna(0)
ems = pd.read_csv('cleaned_ems.csv', na_values=[" ", ""] ,sep=',', index_col=0).fillna(0)
rehab = pd.read_csv('cleaned_rehab.csv', na_values=[" ", ""] ,sep=',', index_col=0).fillna(0)

hospital=hospital.rename(columns = {'Count':'hospital_count'})
dropbox=dropbox.rename(columns = {'COUNT':'dropbox_count'})
dropbox=dropbox.rename(columns = {'24Availability':'dropbox_247availability'})

In [9]:
normalized_data = data.iloc[:,1:]/population.iloc[:,1:]

In [15]:
df = data[['County','Rural']].copy()
df1 = pd.concat([df, rehab['Rehab_Count'], ems['EMS_COUNT'], hospital['hospital_count'], dropbox['dropbox_247availability'], dropbox['dropbox_count']], axis=1)

In [16]:
#data[['Counties','Rural', '2016', '1yrlag', '2yrlag', '10yrlag']]
df1['1yrlag']=(data['2016']-data['2015'])/data['2015']
df1['5yrlag']=(data['2016']-data['2011'])/data['2011']
df1['10yrlag']=(data['2016']-data['2006'])/data['2006']

df1['UR_1yrlag']=(UR['2016']-UR['2015'])/UR['2015']
df1['UR_5yrlag']=(UR['2016']-UR['2011'])/UR['2011']
df1['UR_10yrlag']=(UR['2016']-UR['2006'])/UR['2006']

df1['CR_1yrlag']=(crime['2015']-crime['2014'])/crime['2014']
df1['CR_5yrlag']=(crime['2015']-crime['2010'])/crime['2010']
df1['CR_10yrlag']=(crime['2015']-crime['2005'])/crime['2005']

df1['PR_1yrlag']=(prescription['2016']-prescription['2015'])/prescription['2015']
df1['PR_5yrlag']=(prescription['2016']-prescription['2011'])/prescription['2011']
df1['PR_10yrlag']=(prescription['2016']-prescription['2006'])/prescription['2006']

df1['BR_1yrlag']=(birth_rate['2016']-birth_rate['2015'])/birth_rate['2015']
df1['BR_5yrlag']=(birth_rate['2016']-birth_rate['2011'])/birth_rate['2011']
df1['BR_10yrlag']=(birth_rate['2016']-birth_rate['2006'])/birth_rate['2006']

df1['DR_1yrlag']=(death_rate['2016']-death_rate['2015'])/death_rate['2015']
df1['DR_5yrlag']=(death_rate['2016']-death_rate['2011'])/death_rate['2011']
df1['DR_10yrlag']=(death_rate['2016']-death_rate['2006'])/death_rate['2006']

df1=df1.replace([np.inf, -np.inf], np.nan)
df1 = df1.fillna(0)

df2 = pd.concat([df1, normalized_data['2016']], axis=1)
df2=df2.rename(columns = {'2016':'2016_ODRate'})

In [17]:
df2.head(10)

,County,Rural,Rehab_Count,EMS_COUNT,hospital_count,dropbox_247availability,dropbox_count,1yrlag,5yrlag,10yrlag,...,PR_1yrlag,PR_5yrlag,PR_10yrlag,BR_1yrlag,BR_5yrlag,BR_10yrlag,DR_1yrlag,DR_5yrlag,DR_10yrlag,2016_ODRate
0,Appling,1,0,1,0,1,1,0.000000,0.000000,0.000000,...,-0.044788,0.172512,0.391451,-0.037731,-0.040757,-0.032132,0.024701,0.100460,-0.007407,0.000163
1,Atkinson,1,0,0,0,0,0,0.000000,0.000000,0.000000,...,-0.128370,-0.049020,0.374494,0.177560,0.094003,-0.155060,0.320503,-0.070309,-0.339263,0.000121
2,Bacon,1,0,1,0,0,1,0.000000,-1.000000,-1.000000,...,0.041667,-0.050060,0.080988,-0.148873,0.033370,-0.175538,-0.016046,-0.212045,-0.106060,0.000000
3,Baker,1,0,1,0,0,1,-1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.095897,-0.021526,0.292040,0.017116,0.070206,0.165342,0.000000
4,Banks,1,1,1,2,1,1,0.333333,-0.333333,3.000000,...,-0.042802,-0.180000,0.157647,-0.018961,-0.111911,-0.168930,0.081872,0.224818,0.135277,0.000089
5,Ben Hill,1,0,1,0,0,1,0.000000,0.000000,0.000000,...,-0.118750,0.081841,5.042857,-0.021191,0.189421,-0.114650,-0.096894,0.136966,-0.030297,0.000054
6,Berrien,1,1,1,1,0,2,0.000000,0.000000,-0.666667,...,-0.035348,-0.073384,-0.080435,-0.003088,-0.082432,-0.219392,0.047608,0.046655,0.178995,0.000013
7,Bleckley,1,1,1,1,1,2,-0.666667,0.000000,0.000000,...,-0.016252,-0.125743,0.055385,-0.038482,-0.014972,-0.251839,-0.008083,0.028507,0.125486,0.000010
8,Brantley,1,0,1,1,1,1,0.400000,0.000000,2.500000,...,-0.041807,0.437635,0.495874,-0.027279,-0.083199,-0.286650,0.010204,0.096098,0.176910,0.000406
9,Brooks,1,1,1,1,0,1,0.000000,-0.500000,0.000000,...,-0.020942,-0.418653,-0.439560,-0.009115,0.197646,-0.084032,0.096147,0.085371,0.148933,0.000053


In [24]:
nat_OD_rate=0.00016219127
dangerous = df2[df2['2016_ODRate']>nat_OD_rate]
ok = df2[df2['2016_ODRate']<=nat_OD_rate]

In [25]:
ok

,County,Rural,Rehab_Count,EMS_COUNT,hospital_count,dropbox_247availability,dropbox_count,1yrlag,5yrlag,10yrlag,...,PR_1yrlag,PR_5yrlag,PR_10yrlag,BR_1yrlag,BR_5yrlag,BR_10yrlag,DR_1yrlag,DR_5yrlag,DR_10yrlag,2016_ODRate
1,Atkinson,1,0,0,0,0,0,0.000000,0.000000,0.000000,...,-0.128370,-0.049020,0.374494,0.177560,0.094003,-0.155060,0.320503,-0.070309,-0.339263,0.000121
2,Bacon,1,0,1,0,0,1,0.000000,-1.000000,-1.000000,...,0.041667,-0.050060,0.080988,-0.148873,0.033370,-0.175538,-0.016046,-0.212045,-0.106060,0.000000
3,Baker,1,0,1,0,0,1,-1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.095897,-0.021526,0.292040,0.017116,0.070206,0.165342,0.000000
4,Banks,1,1,1,2,1,1,0.333333,-0.333333,3.000000,...,-0.042802,-0.180000,0.157647,-0.018961,-0.111911,-0.168930,0.081872,0.224818,0.135277,0.000089
5,Ben Hill,1,0,1,0,0,1,0.000000,0.000000,0.000000,...,-0.118750,0.081841,5.042857,-0.021191,0.189421,-0.114650,-0.096894,0.136966,-0.030297,0.000054
6,Berrien,1,1,1,1,0,2,0.000000,0.000000,-0.666667,...,-0.035348,-0.073384,-0.080435,-0.003088,-0.082432,-0.219392,0.047608,0.046655,0.178995,0.000013
7,Bleckley,1,1,1,1,1,2,-0.666667,0.000000,0.000000,...,-0.016252,-0.125743,0.055385,-0.038482,-0.014972,-0.251839,-0.008083,0.028507,0.125486,0.000010
9,Brooks,1,1,1,1,0,1,0.000000,-0.500000,0.000000,...,-0.020942,-0.418653,-0.439560,-0.009115,0.197646,-0.084032,0.096147,0.085371,0.148933,0.000053
10,Bryan,1,1,1,7,0,1,-1.000000,0.000000,0.000000,...,-0.058357,-0.209794,-0.079577,0.019643,-0.003838,-0.070521,0.026755,0.100574,0.062976,0.000000
13,Calhoun,1,0,1,1,0,1,0.000000,0.000000,0.000000,...,0.032917,-0.202454,0.202114,-0.037503,-0.066881,-0.190308,0.015330,0.180746,0.274863,0.000064


In [169]:
features = list(df2.columns[1:-1])
X = df2[features]
Y = df2['2016_ODRate']

In [170]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [171]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, Y_train)

In [172]:
Y_pred = clf.predict(X_test)

In [178]:
df2.to_csv('AggredgatedData.csv')

In [177]:
Y_test

78     0.000000
155    0.001256
128    0.000197
55     0.000018
94     0.000072
29     0.000000
147    0.000839
51     0.000157
98     0.000000
141    0.000747
19     0.000019
60     0.000135
15     0.000013
65     0.000235
24     0.000017
30     0.000007
126    0.000350
101    0.000037
96     0.000173
16     0.000000
151    0.004232
18     0.000158
12     0.000218
9      0.000053
31     0.000146
125    0.001173
95     0.000149
56     0.000083
145    0.000250
152    0.000376
135    0.001879
76     0.000000
75     0.000007
138    0.003248
2      0.000000
86     0.000000
45     0.000073
42     0.000075
68     0.000000
118    0.001027
26     0.000161
137    0.001103
146    0.000155
90     0.000000
66     0.000000
36     0.000000
82     0.000526
22     0.000015
85     0.000000
81     0.000000
112    0.000261
11     0.000308
109    0.000083
Name: 2016_ODRate, dtype: float64